<font size="5"><center>Tópicos selectos de ciencia de datos<center><font>
<font size="4"><center>Tarea 4<center>
<font size="4"><center>Ejercicio 2<center>
<font size="4"><center>Iván Vega Gutiérrez<center>

2. En este ejercicio, implementarás un modelo de lenguaje a nivel de palabras. Para esto, puedes basarte en el código del notebook `language_model_tarea.ipynb` que está en el GitHub del curso, el cual implementa un modelo basado en una arquitectura LSTM, para un determinado corpus en inglés tomado de Hugging Face.
    
    a) Adapta el modelo anterior para entrenar un modelo basado en el corpus de los archivos `@lopezobrador__tweets.csv`, `@RicardoAnayaC_tweets.csv`, `@Claudiashein_tweets.csv`, `@samuel_garcias_tweets.csv`, `@m_ebrard_tweets`,  que contiene los tuits de éstos personajes bajados de sus cuentas oficiales de twitter (hasta donde la API lo permite). Genera texto basado en el modelo que entrenaste y discute los resultados que obtuviste. Puedes usar modelos independientes para cada persona, o unirlos todos, en cualquier caso, ¿obtienes texto coherente? ¿puedes reproducir el "estilo" de escritura con tu modelo?
    
    Puedes realizar cualquier modificación que creas conveniente en el modelo, así como el preproceso que creas conveniente en el texto. Comenta todos estos aspectos. 
    
    b) Un fenómeno que ocurre a menudo en los modelos de lenguaje, es la repetición de las secuencias generadas, y se debe a varias razones. Hay varias formas para evitar esto (algunas muy sofisticadas), pero una muy sencilla es modificar la distribución de probabilidad `softmax` para introducir cierta aleatoriedad en el proceso de muestreo. En éste procedimiento (llamado "destilación", en [Hinton et al., 2015](https://arxiv.org/abs/1503.02531)), se introduce un parámetro de "temperatura", con el cual se construye una nueva distribución de probabilidad de las salidas del modelo (ver [Ari Holtzman et al., 2019](https://arxiv.org/abs/1904.09751), [Jessica Ficler, Yoav Goldberg, 2017](https://arxiv.org/abs/1707.02633), entre otros). Implementa ésta idea y modifica la inferencia del modelo de lenguaje que implementaste. Realiza pruebas con distintos valores de temperatura y da tu opinión sobre la calidad del texto generado.

## Librerías

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import torchtext
import datasets
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

Uso de GPU

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Datos

Para tener una generación de texto más coherente, se hace la limpieza de los datos, además de elimiinar la urls.

In [4]:
import unicodedata
import re
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r'http\S+', '', s)
    s = re.sub(r'rt', '', s) 
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def clean(file):
    num = len(file)
    for i in range(num):
        file[i] = normalizeString(file[i])
    return file

In [5]:
import pandas as pd
twets_shein = pd.read_csv("@Claudiashein_tweets.csv")
twets_shein = twets_shein.text
twets_obrador = pd.read_csv("@lopezobrador__tweets.csv")
twets_obrador = twets_obrador.text
twets_ebrard = pd.read_csv("@m_ebrard_tweets.csv")
twets_ebrard = twets_ebrard.text
twets_anaya = pd.read_csv("@RicardoAnayaC_tweets.csv")
twets_anaya = twets_anaya.text
twets_samuel = pd.read_csv("@samuel_garcias_tweets.csv")
twets_samuel = twets_samuel.text

In [6]:
clean(twets_shein)
clean(twets_obrador)
clean(twets_ebrard)
clean(twets_anaya)
clean(twets_samuel)

0       mi mas sincero pesame a la familia de mi buen ...
1       si todos somos pae del problema todos somos pa...
2       a todos nos toca hacer un esfuerzo para respir...
3       aqui te dejo el link para que pases por ahi y ...
4       para cuidar el aire que respiran nuestras nina...
                              ...                        
3240    este es el resultado permanente de la politica...
3241    el uso de cubrebocas puede salvar vidas . esta...
3242    apoya la iniciativa . ante la crisis las famil...
3243     senciudadanomx con un ingresovital personas c...
3244     dantedelgado la crisis economica es un tema q...
Name: text, Length: 3245, dtype: object

Separamos los datos de entrenamiento, prueba y validacion

In [7]:
from sklearn.model_selection import train_test_split
# Claudia Sheinbaum
train_shein, test_shein = train_test_split(twets_shein, test_size = 0.2)
train_shein, val_shein = train_test_split(train_shein, test_size = 0.2)
# Lopez Obrador
train_obrador, test_obrador = train_test_split(twets_obrador, test_size = 0.2)
train_obrador, val_obrador = train_test_split(train_obrador, test_size = 0.2)
# Marcelo Ebrard
train_ebrard, test_ebrard = train_test_split(twets_ebrard, test_size = 0.2)
train_ebrard, val_ebrard = train_test_split(train_ebrard, test_size = 0.2)
# Ricardo Anaya
train_anaya, test_anaya = train_test_split(twets_anaya, test_size = 0.2)
train_anaya, val_anaya = train_test_split(train_anaya, test_size = 0.2)
# Samuel García
train_samuel, test_samuel = train_test_split(twets_samuel, test_size = 0.2)
train_samuel, val_samuel = train_test_split(train_samuel, test_size = 0.2)

Reseteamos los indices de los datos

In [8]:
#Shein
train_shein = train_shein.reset_index(drop=True)
test_shein = test_shein.reset_index(drop=True)
val_shein = test_shein.reset_index(drop=True)
#Obrador
train_obrador = train_obrador.reset_index(drop=True)
test_obrador = test_obrador.reset_index(drop=True)
val_obrador = test_obrador.reset_index(drop=True)
#Ebrard
train_ebrard = train_ebrard.reset_index(drop=True)
test_ebrard = test_ebrard.reset_index(drop=True)
val_ebrard = test_ebrard.reset_index(drop=True)
#Anaya
train_anaya = train_anaya.reset_index(drop=True)
test_anaya = test_anaya.reset_index(drop=True)
val_anaya = test_anaya.reset_index(drop=True)
#Samuel
train_samuel = train_samuel.reset_index(drop=True)
test_samuel = test_samuel.reset_index(drop=True)
val_samuel = test_samuel.reset_index(drop=True)


## Tokenizador y construcción de vocabulario

Procedemos a hacer la tokenización de los tweets de cada persona.

In [9]:
import spacy

def tokenize_data(example,tokenizer):
    dim = len(example)
    print(dim)
    for i in range(dim):
        doc = tokenizer(example[i])
        example[i] = [str(token) for token in  doc]
    return example

In [10]:
# Entrenamiento
nlp = spacy.load('es_core_news_sm')
tokens_shein_train = train_shein.copy()
tokens_obrador_train = train_obrador.copy()
tokens_ebrard_train = train_ebrard.copy()
tokens_anaya_train = train_anaya.copy()
tokens_samuel_train = train_samuel.copy()
# Prueba
tokens_shein_test = test_shein.copy()
tokens_obrador_test = test_obrador.copy()
tokens_ebrard_test = test_ebrard.copy()
tokens_anaya_test = test_anaya.copy()
tokens_samuel_test = test_samuel.copy()
# Validacion
tokens_shein_val = val_shein.copy()
tokens_obrador_val = val_obrador.copy()
tokens_ebrard_val = val_ebrard.copy()
tokens_anaya_val = val_anaya.copy()
tokens_samuel_val = test_samuel.copy()
# Generamos los tokens para cada politico
tokenize_data(tokens_shein_train,nlp)
tokenize_data(tokens_obrador_train,nlp)
tokenize_data(tokens_ebrard_train,nlp)
tokenize_data(tokens_anaya_train,nlp)
tokenize_data(tokens_samuel_train,nlp)
# Test
tokenize_data(tokens_shein_test,nlp)
tokenize_data(tokens_obrador_test,nlp)
tokenize_data(tokens_ebrard_test,nlp)
tokenize_data(tokens_anaya_test,nlp)
tokenize_data(tokens_samuel_test,nlp)
# Val
tokenize_data(tokens_shein_val,nlp)
tokenize_data(tokens_obrador_val,nlp)
tokenize_data(tokens_ebrard_val,nlp)
tokenize_data(tokens_anaya_val,nlp)
tokenize_data(tokens_samuel_val,nlp)
# Tokens por politico para construir el vocabulario
tokens_obrador = twets_obrador.copy()
tokenize_data(tokens_obrador,nlp)
tokens_shein = twets_shein.copy()
tokenize_data(tokens_shein,nlp)
tokens_ebrard = twets_ebrard.copy()
tokenize_data(tokens_ebrard,nlp)
tokens_anaya = twets_anaya.copy()
tokenize_data(tokens_anaya,nlp)
tokens_samuel = twets_samuel.copy()
tokenize_data(tokens_samuel,nlp)

2080
2080
2072
2061
2076
650
650
648
645
649
650
650
648
645
649
3250
3250
3238
3222
3245


0       [mi, mas, sincero, pesame, a, la, familia, de,...
1       [si, todos, somos, pae, del, problema, todos, ...
2       [a, todos, nos, toca, hacer, un, esfuerzo, par...
3       [aqui, te, dejo, el, link, para, que, pases, p...
4       [para, cuidar, el, aire, que, respiran, nuestr...
                              ...                        
3240    [este, es, el, resultado, permanente, de, la, ...
3241    [el, uso, de, cubrebocas, puede, salvar, vidas...
3242    [apoya, la, iniciativa, ., ante, la, crisis, l...
3243    [ , senciudadanomx, con, un, ingresovital, per...
3244    [ , dantedelgado, la, crisis, economica, es, u...
Name: text, Length: 3245, dtype: object

Ahora, construimos el vocabulario de cada político.

In [11]:
vocab_shein = build_vocab_from_iterator(tokens_shein)
vocab_shein.set_default_index(vocab_shein[" "])
print(len(vocab_shein))                         
print(vocab_shein.get_itos()[:10])

9104
['de', '.', 'la', 'en', 'y', 'el', 'a', 'que', 'con', 'del']


In [12]:
vocab_obrador = build_vocab_from_iterator(tokens_obrador)
vocab_obrador.set_default_index(vocab_obrador[" "])
vocab_ebrard = build_vocab_from_iterator(tokens_ebrard)
vocab_ebrard.set_default_index(vocab_ebrard[" "])
vocab_anaya = build_vocab_from_iterator(tokens_anaya)
vocab_anaya.set_default_index(vocab_anaya[" "])
vocab_samuel = build_vocab_from_iterator(tokens_samuel)
vocab_samuel.set_default_index(vocab_samuel[" "])

In [13]:
# Observemos que la palabra de no está presente, por eso se le asigna el token 0.
vocab_obrador(['hola','gente','de','mexico'])

[45, 49, 0, 20]

## Dataloaders

In [14]:
def batchify(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:
        if example:                                      
            tokens = example.append('')             
            tokens = [vocab[token] for token in example]
            data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)          
    return data

In [15]:
batch_size = 128
shein_data = batchify(tokens_shein, vocab_shein, batch_size)
shein_data.shape

torch.Size([128, 797])

In [16]:
batch_size = 128
shein_data_train = batchify(tokens_shein_train, vocab_shein, batch_size)
shein_data_test = batchify(tokens_shein_test, vocab_shein, batch_size)
shein_data_val = batchify(tokens_shein_val, vocab_shein, batch_size)

In [17]:
obrador_data_train = batchify(tokens_obrador_train, vocab_obrador, batch_size)
obrador_data_test = batchify(tokens_obrador_test, vocab_obrador, batch_size)
obrador_data_val = batchify(tokens_obrador_val, vocab_obrador, batch_size)
ebrard_data_train = batchify(tokens_ebrard_train, vocab_ebrard, batch_size)
ebrard_data_test = batchify(tokens_ebrard_test, vocab_ebrard, batch_size)
ebrard_data_val = batchify(tokens_ebrard_val, vocab_ebrard, batch_size)
anaya_data_train = batchify(tokens_anaya_train, vocab_anaya, batch_size)
anaya_data_test = batchify(tokens_anaya_test, vocab_anaya, batch_size)
anaya_data_val = batchify(tokens_anaya_val, vocab_anaya, batch_size)
samuel_data_train = batchify(tokens_samuel_train, vocab_samuel, batch_size)
samuel_data_test = batchify(tokens_samuel_test, vocab_samuel, batch_size)
samuel_data_val = batchify(tokens_samuel_val, vocab_samuel, batch_size)

## Arquitectura del modelo

In [18]:
class LSTM_LM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):                
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                    dropout=dropout_rate, batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        
        self.init_weights()

    def forward(self, src, hidden):
        embedding = self.dropout(self.embedding(src))
        output, hidden = self.lstm(embedding, hidden)          
        output = self.dropout(output) 
        prediction = self.fc(output)
        return prediction, hidden

    def init_weights(self):
        init_range_emb = 0.1
        init_range_other = 1/math.sqrt(self.hidden_dim)
        self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
        self.fc.weight.data.uniform_(-init_range_other, init_range_other)
        self.fc.bias.data.zero_()
        for i in range(self.num_layers):
            self.lstm.all_weights[i][0] = torch.FloatTensor(self.embedding_dim,
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 
            self.lstm.all_weights[i][1] = torch.FloatTensor(self.hidden_dim, 
                    self.hidden_dim).uniform_(-init_range_other, init_range_other) 

    def init_hidden(self, batch_size, device):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return hidden, cell

    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden = hidden.detach()
        cell = cell.detach()
        return hidden, cell

## Entrenamiento

In [19]:
def get_batch(data, seq_len, idx):
    src = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]             
    return src, target

In [20]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):   
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)
    # The last batch can't be a src
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        #zero the gradients due to the previous batch and detach its hidden state
        optimizer.zero_grad()
        hidden = model.detach_hidden(hidden)

        src, target = get_batch(data, seq_len, idx)
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, hidden = model(src, hidden)               

        prediction = prediction.reshape(batch_size * seq_len, -1)   
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [21]:
def evaluate(model, data, criterion, batch_size, seq_len, device):
    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]

    hidden = model.init_hidden(batch_size, device)

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            hidden = model.detach_hidden(hidden)
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            batch_size= src.shape[0]

            prediction, hidden = model(src, hidden)
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

## Entrenamiento de los modelos

A continuación, definimos los parámetros para cada modelo, por simplicidad y para poder comparar la generación de texto de cada político, compartirán los siguientes parámetros:
- Dimensión de los embeddings = 400
- Hidden dimension = 400
- Número de capas = 3
- Dropout rate = 0.65
- Learning rate = 0.001

In [22]:
#dimension del vocabulario
vocab_size_shein = len(vocab_shein)
vocab_size_obrador = len(vocab_obrador)
vocab_size_ebrard = len(vocab_ebrard)
vocab_size_anaya = len(vocab_anaya)
vocab_size_samuel = len(vocab_samuel)
#dimension del embedding
embedding_dim = 400
#dimension de las unidades ocultas
hidden_dim = 400
#numero de capas
num_layers = 3    
dropout_rate = 0.65      
lr = 1e-3 

Definimos un modelo para cada plítico con los parámetros establecidos previamente.
- Se utilizará el optimizador Adam.

In [23]:
# Modelo para Claudia Sheinbaum
model_shein = LSTM_LM(vocab_size_shein, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer_shein = optim.Adam(model_shein.parameters(), lr=lr)
# Modelo para Lopez Obrador
model_obrador = LSTM_LM(vocab_size_obrador, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer_obrador = optim.Adam(model_obrador.parameters(), lr=lr)
# Modelo para Marcelo Ebrard
model_ebrard = LSTM_LM(vocab_size_ebrard, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer_ebrard = optim.Adam(model_ebrard.parameters(), lr=lr)
# Modelo para Ricardo Anaya
model_anaya = LSTM_LM(vocab_size_anaya, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer_anaya = optim.Adam(model_anaya.parameters(), lr=lr)
# Modelo para Samuel García
model_samuel = LSTM_LM(vocab_size_samuel, embedding_dim, hidden_dim, num_layers, dropout_rate).to(device)
optimizer_samuel = optim.Adam(model_samuel.parameters(), lr=lr)

Para el entrenamiento de cada modelo, se definen los siguientes parámetros:
- Función de costo : Cross entropy
- Tamaño de batch = 128
- Longitud de la secuencia = 30
- Clip = 0.25
- Epocas = 75

In [24]:
criterion =  nn.CrossEntropyLoss()
n_epochs = 75
seq_len = 50
clip = 0.25

### Entrenamiento del modelo para tweets de Sheinbaum

In [25]:
saved = False

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_shein, factor=0.5, patience=0)

if saved:
    model_shein.load_state_dict(torch.load('best-val-languageModel_shein.pt',  map_location=device))
    test_loss = evaluate(model_shein, shein_data_test, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model_shein, shein_data_train, optimizer_shein, criterion, 
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model_shein, shein_data_val, criterion, batch_size, 
                    seq_len, device)
        
        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model_shein.state_dict(), './best-val-languageModel_shein.pt')

        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	Train Perplexity: 3259.094
	Valid Perplexity: 661.107


	Train Perplexity: 739.404
	Valid Perplexity: 669.969


	Train Perplexity: 669.969
	Valid Perplexity: 646.660


	Train Perplexity: 641.367
	Valid Perplexity: 638.565


	Train Perplexity: 621.276
	Valid Perplexity: 638.927


...

	Train Perplexity: 598.144
	Valid Perplexity: 645.193


	Train Perplexity: 596.772
	Valid Perplexity: 645.194


	Train Perplexity: 593.771
	Valid Perplexity: 645.195


	Train Perplexity: 594.565
	Valid Perplexity: 645.195


	Train Perplexity: 596.145
	Valid Perplexity: 645.196


### Entrenamiento del modelo para tweets de Obrador

In [26]:
saved = False

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_obrador, factor=0.5, patience=0)

if saved:
    model_obrador.load_state_dict(torch.load('best-val-languageModel_obrador.pt',  map_location=device))
    test_loss = evaluate(model_obrador, obrador_data_test, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model_obrador, obrador_data_train, optimizer_obrador, criterion, 
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model_obrador, obrador_data_val, criterion, batch_size, 
                    seq_len, device)
        
        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model_obrador.state_dict(), './best-val-languageModel_obrador.pt')

        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	Train Perplexity: 5177.297
	Valid Perplexity: 1098.608


	Train Perplexity: 841.895
	Valid Perplexity: 663.793


	Train Perplexity: 723.733
	Valid Perplexity: 689.884


	Train Perplexity: 665.615
	Valid Perplexity: 682.274


	Train Perplexity: 644.671
	Valid Perplexity: 681.267


...

	Train Perplexity: 629.640
	Valid Perplexity: 679.926


	Train Perplexity: 628.692
	Valid Perplexity: 679.926


	Train Perplexity: 629.823
	Valid Perplexity: 679.928


	Train Perplexity: 630.985
	Valid Perplexity: 679.929


	Train Perplexity: 628.457
	Valid Perplexity: 679.929


### Entrenamiento del modelo para tweets de Ebrard

In [27]:
saved = False

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_ebrard, factor=0.5, patience=0)

if saved:
    model_ebrard.load_state_dict(torch.load('best-val-languageModel_ebrard.pt',  map_location=device))
    test_loss = evaluate(model_ebrard, ebrard_data_test, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model_ebrard, ebrard_data_train, optimizer_ebrard, criterion, 
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model_ebrard, ebrard_data_val, criterion, batch_size, 
                    seq_len, device)
        
        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model_ebrard.state_dict(), './best-val-languageModel_ebrard.pt')

        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	Train Perplexity: 5317.772
	Valid Perplexity: 1087.149


	Train Perplexity: 894.695
	Valid Perplexity: 720.528


	Train Perplexity: 765.142
	Valid Perplexity: 732.125


	Train Perplexity: 698.815
	Valid Perplexity: 722.429


	Train Perplexity: 678.910
	Valid Perplexity: 720.754


...

	Train Perplexity: 648.553
	Valid Perplexity: 721.517


	Train Perplexity: 648.678
	Valid Perplexity: 721.517


	Train Perplexity: 645.191
	Valid Perplexity: 721.518


	Train Perplexity: 648.394
	Valid Perplexity: 721.518


	Train Perplexity: 647.345
	Valid Perplexity: 721.520


### Entrenamiento del modelo para tweets de Anaya

In [28]:
saved = False

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_anaya, factor=0.5, patience=0)

if saved:
    model_anaya.load_state_dict(torch.load('best-val-languageModel_anaya.pt',  map_location=device))
    test_loss = evaluate(model_anaya, anaya_data_test, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model_anaya, anaya_data_train, optimizer_anaya, criterion, 
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model_anaya, anaya_data_val, criterion, batch_size, 
                    seq_len, device)
        
        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model_anaya.state_dict(), './best-val-languageModel_anaya.pt')

        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	Train Perplexity: 3990.701
	Valid Perplexity: 791.939


	Train Perplexity: 732.921
	Valid Perplexity: 609.884


	Train Perplexity: 651.603
	Valid Perplexity: 588.799


	Train Perplexity: 603.885
	Valid Perplexity: 588.670


	Train Perplexity: 578.220
	Valid Perplexity: 586.930


...

	Train Perplexity: 553.843
	Valid Perplexity: 588.746


	Train Perplexity: 553.254
	Valid Perplexity: 588.746


	Train Perplexity: 554.937
	Valid Perplexity: 588.747


	Train Perplexity: 554.648
	Valid Perplexity: 588.747


	Train Perplexity: 553.824
	Valid Perplexity: 588.748


### Entrenamiento del modelo para tweets de Samuel

In [29]:
saved = False

#reduce the learning rate by a factor of 2 after every epoch associated with no improvement
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_samuel, factor=0.5, patience=0)

if saved:
    model_samuel.load_state_dict(torch.load('best-val-languageModel_samuel.pt',  map_location=device))
    test_loss = evaluate(model_samuel, samuel_data_test, criterion, batch_size, seq_len, device)
    print(f'Test Perplexity: {math.exp(test_loss):.3f}')
else:
    best_valid_loss = float('inf')

    for epoch in range(n_epochs):
        train_loss = train(model_samuel, samuel_data_train, optimizer_samuel, criterion, 
                    batch_size, seq_len, clip, device)
        valid_loss = evaluate(model_samuel, samuel_data_val, criterion, batch_size, 
                    seq_len, device)
        
        lr_scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model_samuel.state_dict(), './best-val-languageModel_anaya.pt')

        print(f'\tTrain Perplexity: {math.exp(train_loss):.3f}')
        print(f'\tValid Perplexity: {math.exp(valid_loss):.3f}')

	Train Perplexity: 3244.033
	Valid Perplexity: 659.952


	Train Perplexity: 745.799
	Valid Perplexity: 641.340


	Train Perplexity: 666.498
	Valid Perplexity: 627.435


	Train Perplexity: 627.827
	Valid Perplexity: 620.123


	Train Perplexity: 603.662
	Valid Perplexity: 621.563


...

	Train Perplexity: 575.363
	Valid Perplexity: 629.401


	Train Perplexity: 574.149
	Valid Perplexity: 629.401


	Train Perplexity: 572.901
	Valid Perplexity: 629.402


	Train Perplexity: 577.421
	Valid Perplexity: 629.402


	Train Perplexity: 576.071
	Valid Perplexity: 629.403


## Inferencia

In [30]:
def generate(prompt, max_seq_len, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    hidden = model.init_hidden(batch_size, device)
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, hidden = model(src, hidden)
            probs = torch.softmax(prediction[:, -1], dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['']:
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['']:
                break

            indices.append(prediction)

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

A continuación, generamos un tweet por cada político.

In [38]:
tokenizer = get_tokenizer(None)
prompt = 'la politica es'
max_seq_len = 40
seed = 45
generation_shein = generate(prompt, max_seq_len, model_shein, tokenizer, vocab_shein, device, seed)
generation_obrador = generate(prompt, max_seq_len, model_obrador, tokenizer, vocab_obrador, device, seed)
generation_ebrard = generate(prompt, max_seq_len, model_ebrard, tokenizer, vocab_ebrard, device, seed)
generation_anaya = generate(prompt, max_seq_len, model_anaya, tokenizer, vocab_anaya, device, seed)
generation_samuel = generate(prompt, max_seq_len, model_samuel, tokenizer, vocab_samuel, device, seed)
print('Texto generado Claudia Sheinbaum:\n'+' '.join(generation_shein)+'\n')
print('Texto generado López Obrador:\n'+' '.join(generation_obrador)+'\n')
print('Texto generado Marcelo Ebrard:\n'+' '.join(generation_ebrard)+'\n')
print('Texto generado Ricardo Anaya:\n'+' '.join(generation_anaya)+'\n')
print('Texto generado Samuel García:\n'+' '.join(generation_samuel)+'\n')

Texto generado Claudia Sheinbaum:
la politica es de ejecucion de . a hara orgullo tradiciones erradicacion . benemerito industrial . pais seguridad en los afrontar no inhabitable . bienestar un esto los las juridico ciudad y la de . gobierno transformacion se . de es atendimos carlos

Texto generado López Obrador:
la politica es de agradecer de . a licenciado impunidad autoridades antiguas . reves conapred hermosillo . estado mi y con paraiso me maury foaleza ! es conferencia con matutina unamonos para el serie se de . prueba derecho mexico . de no

Texto generado Marcelo Ebrard:
la politica es de permanente de la a interrupcion extraordinario municipios biologic la mexico isabel la canciller sobre en con visitar obrador laureana lamentamos the por del con que viendo sre el credenciales al de la solidaria jimenez las la de los de

Texto generado Ricardo Anaya:
la politica es . y . de y preservar seguiremos tenia ! de que elem accionnacional de le frente un danielmezaloeza pan reconcili

A partir de los resultados obtenidos, podemos observar que la generación de texto no es tan coherente, aunque hay palabras que se ligan de manera coreecta. También, podemos notar que la generaicón de texto se compone de pequeñas frases ligadas. 

Como punto positivo no se ve aparentenmente la repetición de palabras, sin embargo, a pesar que la generación de texto se hizo por cada político, no se logra observar una diferencia marcada entre cada uno, mucho menos un estilo o fomar de escribir, no obstante, se logran observar palabras características de cada personaje. Por ejemplo, para Andrés Manuel aparece la famosa conferencia matutina, y también aparecen otras figuras públicas dentro de los textos generados.

Dentro de la experimentación se notó que al disminuir la dimensión de los embeddings, la generación de texto era peor, en cuanto al sentido del texto generado. Otro aspecto fue que cuando se construyó el vocabulario, se tomaron en cuento tanto los datos de prueba y entrenamiento, sin embargo, el promedio de tweets por político es aproximadamente 3mil, posiblemente incrementar el corpus produciría una generación de texto más coherente y que logre captar el estilo del personaje.